In [2]:
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from vol.vol import Heston

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg, european_call_payoff
from hestonmc_cuda import mc_price_cupy, simulate_heston_euler_cupy, simulate_heston_andersen_qe_cupy, simulate_heston_andersen_tg_cupy, european_call_payoff_cupy

In [3]:
heston_params_1 = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3) #from stoch vol
heston_params_2 = HestonParameters(kappa = 1, gamma = 0.4, rho = -0.1, vbar = 0.2, v0 = 0.2) # from school
heston_params_3 = HestonParameters(kappa = 0.5, gamma = 1, rho = -0.9, vbar = 0.04, v0 = 0.04) #  from andeson paper 1
heston_params_4 = HestonParameters(kappa = 0.3, gamma = 0.9, rho = -0.5, vbar = 0.04, v0 = 0.04) #  from andeson paper 2
heston_params_5 = HestonParameters(kappa = 1, gamma = 1, rho = -0.3, vbar = 0.04, v0 = 0.09) #  from andeson paper 3

heston_params_array = [heston_params_1, heston_params_2, heston_params_3, heston_params_4, heston_params_5]

state         = MarketState(stock_price = 1.*100, interest_rate = 0.)

r_x           = np.load(r"Data/anderson tg/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = np.load(r"Data/anderson tg/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = np.load(r"Data/anderson tg/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
kwargs        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }
r_x           = cp.load(r"Data/anderson tg/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = cp.load(r"Data/anderson tg/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = cp.load(r"Data/anderson tg/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
kwargs_cupy        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }

In [4]:
import time
from tqdm.contrib.itertools import product

In [1]:
model = Heston(state.stock_price, heston_params_3.v0, heston_params_3.kappa, heston_params_3.vbar, heston_params_3.gamma, heston_params_3.rho, state.interest_rate)
model.call_price(0.4, 70)

NameError: name 'Heston' is not defined

In [ ]:
test_1 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(70, 120, 70),
            'Ts': np.linspace(0.1, 5, 70),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(100_000, 1000_000, 100000_000),
            'heston_params': heston_params_1
          }

MC_compare_models_grid_test_1= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [19]:
test_2 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(70, 120, 20),
            'Ts': np.linspace(0.1, 5, 20),
            'N_Ts': range(50, 125, 500),
            'batch_sizes': range(100_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_2= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [8]:
test_3 = {'schemes':[simulate_heston_andersen_qe_cupy, simulate_heston_andersen_tg_cupy, simulate_heston_euler_cupy],
          'strikes': np.linspace(60, 140, 5),
            'Ts': np.linspace(0.1, 5, 5),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(500_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_3= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [9]:

i = 0

for scheme, strike, T, N_T, batch_size , heston_params_n in product(test_3['schemes'], test_3['strikes'], test_3['Ts'], test_3['N_Ts'], test_3['batch_sizes'], test_3['heston_params_n']):
    heston_params_ = heston_params_array[heston_params_n-1]
    ec_payoff = european_call_payoff_cupy(T, strike, state.interest_rate)
    
    common_mc_params = {"absolute_error": 5e-2, "state": state, "heston_params": heston_params_, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": False}
    model = Heston(state.stock_price, heston_params_.v0, heston_params_.kappa, heston_params_.vbar, heston_params_.gamma, heston_params_.rho, state.interest_rate)
    
    if scheme == simulate_heston_andersen_tg_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params, **kwargs_cupy))
        et = time.time()
    elif scheme == simulate_heston_euler_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params))
        et = time.time()
    elif scheme == simulate_heston_andersen_qe_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params))
        et = time.time()
    MC_compare_models_grid_test_3.loc[i] = (scheme.__name__, heston_params_n, strike, T, N_T, batch_size,common_mc_params['absolute_error'], model.call_price(T, strike), res, et-st)
    i +=1

  0%|          | 0/750 [00:00<?, ?it/s]

/media/kir1200/E4C4BDFBC4BDCFD2/Other/code/jypeter/Stochastic-Volatility-Models/Code/vol/vol/heston.py:96: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  intg.quad(
/media/kir1200/E4C4BDFBC4BDCFD2/Other/code/jypeter/Stochastic-Volatility-Models/Code/vol/vol/heston.py:96: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probab

In [76]:
MC_compare_models_grid_test_2.to_csv(r"Data/evaluation/MC_compare_models_grid_test_2.csv")

In [4]:
MC_compare_models_grid_test_1  = pd.read_csv('Data/evaluation/MC_compare_models_grid_test_1_fixed_true.csv', index_col=0)

In [119]:
#MC_compare_models_grid_test_1.to_csv(r"Data/evaluation/MC_compare_models_grid_test_1_fixed_true.csv")

In [132]:
MC_compare_models_grid_test_1

,scheme,heston_params#,strike,T,N_T,batch_size,absolute error,true,MC_price,time
0,simulate_heston_euler_cupy,1,70.0,0.100000,50,100000,0.05,30.131379,30.129462,0.111071
1,simulate_heston_euler_cupy,1,70.0,0.100000,100,100000,0.05,30.131379,30.143064,0.198346
2,simulate_heston_euler_cupy,1,70.0,0.171014,50,100000,0.05,30.499960,30.483909,0.145198
3,simulate_heston_euler_cupy,1,70.0,0.171014,100,100000,0.05,30.499960,30.480420,0.301842
4,simulate_heston_euler_cupy,1,70.0,0.242029,50,100000,0.05,30.955468,30.929116,0.180836
...,...,...,...,...,...,...,...,...,...,...
19595,simulate_heston_andersen_tg_cupy,1,120.0,4.857971,100,100000,0.05,18.684402,18.809335,1.729174
19596,simulate_heston_andersen_tg_cupy,1,120.0,4.928986,50,100000,0.05,18.811424,19.045366,0.698318
19597,simulate_heston_andersen_tg_cupy,1,120.0,4.928986,100,100000,0.05,18.811424,18.934070,1.756082
19598,simulate_heston_andersen_tg_cupy,1,120.0,5.000000,50,100000,0.05,18.937836,19.157635,0.713773


In [32]:
scheme_  = 'simulate_heston_euler_cupy'
N_T_ = 50
heston_params_n_ = 4

df = MC_compare_models_grid_test_3

test2_euler = MC_compare_models_grid_test_3[df['scheme'] == scheme_] [df['N_T'] == N_T_][df['heston_params#'] == heston_params_n_]

test2_euler = test2_euler[['strike', 'T', 'MC_price', 'true']]




scheme_  = 'simulate_heston_andersen_qe_cupy'

test2_anderson_qe = df[df['scheme'] == scheme_] [df['N_T'] == N_T_][df['heston_params#'] == heston_params_n_]

test2_anderson_qe = test2_anderson_qe[['strike', 'T', 'MC_price', 'true']]




scheme_  = 'simulate_heston_andersen_tg_cupy'

test2_anderson_tg = df[df['scheme'] == scheme_] [df['N_T'] == N_T_][df['heston_params#'] == heston_params_n_]


test2_anderson_tg = test2_anderson_tg[['strike', 'T', 'MC_price', 'true']]

/tmp/ipykernel_34519/1799535111.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_34519/1799535111.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_34519/1799535111.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [33]:
test2_euler['error'] = test2_euler['MC_price'] - test2_euler['true']
test2_anderson_tg['error'] = test2_anderson_tg['MC_price'] - test2_anderson_tg['true']
test2_anderson_qe['error'] = test2_anderson_qe['MC_price'] - test2_anderson_qe['true']

In [35]:
test2_euler = test2_euler.pivot(index='strike', columns='T', values='error')
test2_anderson_tg = test2_anderson_tg.pivot(index='strike', columns='T', values='error')
test2_anderson_qe = test2_anderson_qe.pivot(index='strike', columns='T', values='error')

In [18]:
test2_euler = test2_euler.pivot(index='strike', columns='T', values='true')
test2_anderson_tg = test2_anderson_tg.pivot(index='strike', columns='T', values='true')
test2_anderson_qe = test2_anderson_qe.pivot(index='strike', columns='T', values='true')

KeyError: 'strike'

In [94]:
test2_euler = test2_euler.pivot(index='strike', columns='T', values='MC_price')
test2_anderson_tg = test2_anderson_tg.pivot(index='strike', columns='T', values='MC_price')
test2_anderson_qe = test2_anderson_qe.pivot(index='strike', columns='T', values='MC_price')

In [43]:
test2_euler = test2_euler.abs()
test2_anderson_tg = test2_anderson_tg.abs()
test2_anderson_qe = test2_anderson_qe.abs()

In [44]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'surface'}, {'type':'surface'}, {'type':'surface'}]],)

fig.add_trace(go.Surface(z=test2_euler.values, x=test2_euler.columns, y=test2_euler.index, showscale=False), row=1, col=1, )
fig.add_trace(go.Surface(z=test2_anderson_tg.values, x=test2_anderson_tg.columns, y=test2_anderson_tg.index, showscale=False), row=1, col=2)
fig.add_trace(go.Surface(z=test2_anderson_qe.values, x=test2_anderson_qe.columns, y=test2_anderson_qe.index, showscale=False), row=1, col=3)

fig.show()